# Task 1 ( Fine-Tuning CNN Dailymail Dataset )

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NLP/

/content/drive/MyDrive/NLP


In [ ]:
from THExt-ensemble import finetuning

dataset_fine_tuning_THExt = pd.read_csv("dataset_fine_tuning_THExt.csv") 

finetunig(dataset_fine_tuning_THExt, "checkpoint"

Evaluate

In [ ]:
def evaluate(self, text, hs,  sentences):

    predicted_highlights_concat = ' '.join(map(str, sentences))
    real_highlights_concat =  hs

    r_computer = rouge.Rouge(metrics=['rouge-n', 'rouge-l'], limit_length=False, max_n=2, alpha=0.5, stemming=False)
    score = r_computer.get_scores(predicted_highlights_concat,real_highlights_concat) 

    return score['rouge-1']['f'],score['rouge-2']['f'], score['rouge-l']['f']

In [ ]:
data = load_dataset("cnn_dailymail" ,"3.0.0", split="validation")
data = pd.DataFrame(data).iloc[:1000]

In [ ]:
model_name_or_path='checkpoint' 
base_model_name = "morenolq/thext-cs-scibert"

In [ ]:
sr = SentenceRankerPlus(device='cuda')
sr.load_model(base_model_name=base_model_name, model_name_or_path=model_name_or_path,device='cuda')
rm = RedundancyManager()
h = Highlighter(sr, redundancy_manager = rm)

In [ ]:
r1_f = np.array([])
r2_f = np.array([])
rl_f = np.array([])

In [ ]:
text = data.iloc[i]['article']
highlights = data.iloc[i]['highlights']

sum = h.get_highlights_simple(text, abstract = True, rel_w=1.0, pos_w=0.0, red_w=0.0, prefilter=False, NH = 3)


r1f_ind,r2f_ind,rlf_ind = 
r1f,r2f,rlf = evaluate(text, highlights, sent = sentences)

r1_f = np.append(r1_f,r1f)
r2_f = np.append(r2_f,r2f)
rl_f = np.append(rl_f,rlf)

In [1]:
print(f"Avarage Rougue-1 f1 score : { np.average(r1_f) }")
print(f"Avarage Rougue-2 f1 score : { np.average(r2_f) }")
print(f"Avarage Rougue-l f1 score : { np.average(rl_f) }")

Avarage Rougue-1 f1 score : 0.2228046546371583
Avarage Rougue-2 f1 score : 0.06335872596587086
Avarage Rougue-l f1 score : 0.18059221632888572


# Task 2 ( Ensamble method )

In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
from utils import Ensemble
from datasets import load_dataset
import rouge
import pandas as pd
import numpy as np


Fit models

In [ ]:
from sklearn.linear_model import LassoCV

random_ensamble = Ensemble("RandomForest")
sgd_ensamble = Ensemble("sgd")
lasso_ensamble = Ensemble(model = LassoCV(cv=5, random_state=0))

data = pd.read_csv("data_train.csv") #specificare nel readme di scaricare il dataset dal drive e metterlo nella cartella

X = data[['text_rank', 'lsa_score', 'tf_idf', 'relevance_score', 'thext_score', 'pos_i']]
y = data['rouge_2f']

random_ensamble.train(X,y)
sgd_ensamble.train(X,y)
lasso_ensamble.train(X,y)

random_ensamble.save("random")
sgd_ensamble.save("sgd")
lasso_ensamble.save("lasso")



Test models

In [ ]:
#random_ensamble.load("random")
#sgd_ensamble.load("sgd")
#lasso_ensamble.load("lasso")

data = load_dataset("cnn_dailymail" ,"3.0.0", split="validation")
data = pd.DataFrame(data).iloc[:1000]

In [ ]:
def evaluate(model, data):
	r1_f = []
	r2_f = []
	rl_f = []
	for i in range(len(data)):
		text = data.iloc[i]['article']
		highlights = data.iloc[i]['highlights']
		r1f,r2f,rlf = model.evaluate(text, highlights)
		r1_f.append(r1f)
		r2_f.append(r2f)
		rl_f.append(rlf)
	print(f"Avarage Rougue-1 f1 score : { np.average(r1_f) }")
	print(f"Avarage Rougue-2 f1 score : { np.average(r2_f) }")
	print(f"Avarage Rougue-l f1 score : { np.average(rl_f) }")

In [ ]:
evaluate(random_ensamble, data)

Avarage Rougue-1 f1 score : 0.2996940681666996
Avarage Rougue-2 f1 score : 0.11720662425077215
Avarage Rougue-l f1 score : 0.24533250434085624


In [ ]:
evaluate(sgd_ensamble, data)

Avarage Rougue-1 f1 score : 0.2457166305993262
Avarage Rougue-2 f1 score : 0.07812978592980055
Avarage Rougue-l f1 score : 0.20204450552975334


In [ ]:
evaluate(lasso_ensamble, data)

Avarage Rougue-1 f1 score : 0.2457166305993262
Avarage Rougue-2 f1 score : 0.07812978592980055
Avarage Rougue-l f1 score : 0.20204450552975334


Independent set method

In [ ]:
from utils import RedundancyIndipendentSet

In [ ]:
random_ensamble = Ensemble("RandomForest")

data = load_dataset("cnn_dailymail" ,"3.0.0", split="validation")
data = pd.DataFrame(data).iloc[:1000]

In [ ]:
r = RedundancyIndipendentSet()

r1_f = np.array([])
r2_f = np.array([])
rl_f = np.array([])
r1_f_ind = np.array([])
r2_f_ind = np.array([])
rl_f_ind = np.array([])

In [ ]:
for i in range(len(data)):

  text = data.iloc[i]['article']
  highlights = data.iloc[i]['highlights']

  sum = random_ensamble.summary(text, NH = 3)
  sum_redundancy = random_ensamble.summary(text, NH = 5, score=True)
  ind = r.indipendent_set(sum_redundancy)

  r1f_ind,r2f_ind,rlf_ind = random_ensamble.evaluate(text, highlights, sent = ind )
  r1f,r2f,rlf = random_ensamble.evaluate(text, highlights, sent = sum)

  r1_f = np.append(r1_f,r1f)
  r2_f = np.append(r2_f,r2f)
  rl_f = np.append(rl_f,rlf)
  r1_f_ind = np.append(r1_f_ind,r1f_ind)
  r2_f_ind = np.append(r2_f_ind,r2f_ind)
  rl_f_ind = np.append(rl_f_ind,rlf_ind)

In [ ]:
print(f"\n\nAvarage Rougue-1 f1 score : { np.average(r1_f) }")
print(f"Avarage Rougue-2 f1 score : { np.average(r2_f) }")
print(f"Avarage Rougue-l f1 score : { np.average(rl_f) }")
print(f"\n\nAvarage Rougue-1 f1 score indipendence method : { np.average(r1_f_ind) }")
print(f"Avarage Rougue-2 f1 score  indipendence method : { np.average(r2_f_ind) }")
print(f"Avarage Rougue-l f1 score indipendence method : { np.average(rl_f_ind) }")



Avarage Rougue-1 f1 score : 0.2996940681666996
Avarage Rougue-2 f1 score : 0.11720662425077215
Avarage Rougue-l f1 score : 0.24533250434085624


Avarage Rougue-1 f1 score indipendence method : 0.27656139206147884
Avarage Rougue-2 f1 score  indipendence method : 0.090964688278289
Avarage Rougue-l f1 score indipendence method : 0.22409854793980924
